In [3]:
from django.shortcuts import render
from student.models import Grade, Student, Attendance

#sql to JSON
from django.core import serializers
import jsonpickle

#sql to pandas
import pandas
from django.db import connection

#bokeh
import datetime
from collections import OrderedDict
import numpy as np
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Spectral11, Blues9, Blues5
from bokeh.plotting import figure
from bokeh.charts import Scatter, TimeSeries



#for notebook only
from bokeh.charts import show
from bokeh.io import output_notebook
from bokeh.resources import INLINE

#google
import gviz_api
from gviz_data_table import Table  
import numpy as np
import pandas as pd

In [4]:
attend=Attendance.objects.filter(student_id=2).order_by('-attend_date')[1].calc_pct()
round(attend)


95.0

In [5]:
attend_sql="Select attend_date, absent_days, total_days \
FROM student_attendance WHERE student_id=1"
df_attend = pandas.read_sql(attend_sql, con=connection)
df_attend



,attend_date,absent_days,total_days
0,2015-03-15,1.0,70.0
1,2015-03-30,1.0,85.0
2,2015-04-15,1.5,99.0
3,2015-04-30,1.5,114.0
4,2015-05-15,2.5,129.0
5,2015-05-30,1.5,144.0


In [6]:
attend_dict=df_attend.to_dict(orient="records")

attend_dict

[{'absent_days': 1.0,
  'attend_date': datetime.date(2015, 3, 15),
  'total_days': 70.0},
 {'absent_days': 1.0,
  'attend_date': datetime.date(2015, 3, 30),
  'total_days': 85.0},
 {'absent_days': 1.5,
  'attend_date': datetime.date(2015, 4, 15),
  'total_days': 99.0},
 {'absent_days': 1.5,
  'attend_date': datetime.date(2015, 4, 30),
  'total_days': 114.0},
 {'absent_days': 2.5,
  'attend_date': datetime.date(2015, 5, 15),
  'total_days': 129.0},
 {'absent_days': 1.5,
  'attend_date': datetime.date(2015, 5, 30),
  'total_days': 144.0}]

In [7]:
attend_google_table = Table()
attend_google_table.add_column('attend_date', date, 'Date')
attend_google_table.add_column('absent_days', float, 'Absences')
attend_google_table.add_column('total_days', float, 'Total Days')

for row in df_attend.iterrows():
    attend_google_table.append(row[1].tolist())

attend_google_table.rows

NameError: name 'date' is not defined

In [ ]:
 description = {"year": ("date", "Year"),
                 "sales": ("number", "Sales"),
                 "expenses": ("number", "Expenses")}


# Loading it into gviz_api.DataTable
data_table = gviz_api.DataTable(description)
data_table.LoadData(attend_dict)

In [8]:
attend_sql="Select attend_date, absent_days \
    FROM student_attendance WHERE student_id=1"
df_attend = pandas.read_sql(attend_sql, con=connection)
attend_dict=df_attend.as_matrix()
description2 = [("attend_date", "date" ),
               ("absent_days", "number")]

# Loading it into gviz_api.DataTable
data_table = gviz_api.DataTable(description2)
data_table.LoadData(attend_dict)
json = data_table.ToJSon()
json

'{"rows":[{"c":[{"v":"Date(2015,2,15)"},{"v":1.0}]},{"c":[{"v":"Date(2015,2,30)"},{"v":1.0}]},{"c":[{"v":"Date(2015,3,15)"},{"v":1.5}]},{"c":[{"v":"Date(2015,3,30)"},{"v":1.5}]},{"c":[{"v":"Date(2015,4,15)"},{"v":2.5}]},{"c":[{"v":"Date(2015,4,30)"},{"v":1.5}]}],"cols":[{"type":"date","id":"attend_date","label":"attend_date"},{"type":"number","id":"absent_days","label":"absent_days"}]}'

In [11]:
student_id=4
all_grades_sql = "SELECT grade, grade_date, subject_name  FROM student_grade, student_subject  \
           WHERE student_grade.student_id = '%s' AND student_grade.subject_id=student_subject.subject_id" %(student_id )

df_all_grades = pandas.read_sql(all_grades_sql, con=connection)
df_grades_indexed=df_all_grades.pivot(index='grade_date', columns='subject_name', values='grade')
subject_names=list(df_grades_indexed.columns.values)
df_grades_indexed.insert(0,'date', df_grades_indexed.index)
all_grades_data=df_grades_indexed.values
all_grades_data

# should make this dynamic with a loop!!!
grades_description = [("grade_date", "date", "Date" ),
               ("subj1", "number", subject_names[0] ),
               ("subj2", "number", subject_names[1] ),
               ("subj3", "number", subject_names[2]),
               ("subj4", "number", subject_names[3] )]

# Loading it into gviz_api.DataTable
all_grades_data_table = gviz_api.DataTable(grades_description)
all_grades_data_table.LoadData(all_grades_data)
historical_grades_json = all_grades_data_table.ToJSon()
historical_grades_json


'{"rows":[{"c":[{"v":"Date(2015,2,15)"},{"v":78},{"v":76},{"v":83},{"v":62}]},{"c":[{"v":"Date(2015,3,15)"},{"v":78},{"v":77},{"v":83},{"v":72}]},{"c":[{"v":"Date(2015,4,15)"},{"v":78},{"v":80},{"v":80},{"v":73}]}],"cols":[{"type":"date","id":"grade_date","label":"Date"},{"type":"number","id":"subj1","label":"Math"},{"type":"number","id":"subj2","label":"Reading"},{"type":"number","id":"subj3","label":"Science"},{"type":"number","id":"subj4","label":"Social Studies"}]}'

In [10]:
subject_names[0]

NameError: name 'subject_names' is not defined

In [94]:
for index,subject in enumerate(subject_names):
    print 'subj'+str(index+1), ', number, ', subject

subj1 , number,  Math
subj2 , number,  Reading
subj3 , number,  Science
subj4 , number,  Social Studies


In [37]:
#SQL to dictionary for all grades
all_grades_sql = "SELECT grade, grade_date, subject_name  FROM student_grade, student_subject  \
       WHERE student_grade.student_id = '%s' AND student_grade.subject_id=student_subject.subject_id" %(student_id )
df_all_grades = pandas.read_sql(all_grades_sql, con=connection)
df_grades_indexed=df_all_grades.pivot(index='grade_date', columns='subject_name', values='grade')


subject_names=list(df_grades_indexed.columns.values)
df_grades_indexed.insert(0,'date', df_grades_indexed.index)
all_grades_data=df_grades_indexed.values


grades_desc =[] 
grades_desc.append(("grade_date", "date", "Date" ))
for index,subject in enumerate(subject_names):
    grades_desc.append( ("subj"+str(index+1), "number", subject))

grades_desc
# Loading it into gviz_api.DataTable
all_grades_data_table = gviz_api.DataTable(grades_desc)
all_grades_data_table.LoadData(all_grades_data)
historical_grades_json = all_grades_data_table.ToJSon()
historical_grades_json

'{"rows":[{"c":[{"v":"Date(2015,2,15)"},{"v":78},{"v":76},{"v":83},{"v":62}]},{"c":[{"v":"Date(2015,3,15)"},{"v":78},{"v":77},{"v":83},{"v":72}]},{"c":[{"v":"Date(2015,4,15)"},{"v":78},{"v":80},{"v":80},{"v":73}]}],"cols":[{"type":"date","id":"grade_date","label":"Date"},{"type":"number","id":"subj1","label":"Math"},{"type":"number","id":"subj2","label":"Reading"},{"type":"number","id":"subj3","label":"Science"},{"type":"number","id":"subj4","label":"Social Studies"}]}'